<a href="https://colab.research.google.com/github/DenysDiakiv/AB-testing-toolkit/blob/main/AB-testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Перебіг завдання

Ми почали з того, що агрегували дані для кожної з двох груп (контрольної та тестової) окремо. Потім ми згрупували ці дані за кожною аудиторією — загальною (Total) та специфічними категоріями, такими як країна, пристрій, континент, тощо. Це дозволило нам отримати зведені показники для кожного окремого сегменту.

Після цього ми об’єднали агреговані таблиці для обох груп в одну загальну. До цієї об’єднаної таблиці ми додали кількість сесій для кожного сегмента, що дало нам можливість розрахувати співвідношення між подіями та сесіями. Нарешті, щоб перевірити, чи є різниця між групами значущою, ми провели пропорційний Z-тест, який ідентифікував статистично важливі результати.

In [ ]:
from google.colab import auth
import pandas as pd
from google.cloud import bigquery
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
auth.authenticate_user()

client = bigquery.Client(project='data-analytics-mate')

query = """WITH
  info AS(
  SELECT
    s.date,
    s.ga_session_id,
    p.country,
    p.device,
    p.continent,
    p.channel,
    t.test,
    t.test_group
  FROM
    `DA.ab_test` t
  JOIN
    `DA.session` s
  ON
    t.ga_session_id=s.ga_session_id
  JOIN
    `DA.session_params` p
  ON
    s.ga_session_id= p.ga_session_id),


  total_order AS (
  SELECT
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group,
    COUNT(DISTINCT o.ga_session_id) AS total_order
  FROM
    `DA.order` o
  JOIN
    info i
  ON
    o.ga_session_id=i.ga_session_id
  GROUP BY
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group),


  total_event AS(
  SELECT
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group,
    p.event_name,
    COUNT(p.ga_session_id) total_event
  FROM
    `DA.event_params` p
  JOIN
    info i
  ON
    p.ga_session_id = i.ga_session_id
  GROUP BY
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group,
    p.event_name),


  total_session AS(
  SELECT
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group,
    COUNT( DISTINCT i.ga_session_id) total_session
  FROM
    info i
  GROUP BY
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group),


  total_account AS(
  SELECT
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group,
    COUNT(DISTINCT a.ga_session_id) total_account
  FROM
    `DA.account_session` a
  JOIN
    info i
  ON
    a.ga_session_id =i.ga_session_id
  GROUP BY
    i.date,
    i.country,
    i.device,
    i.continent,
    i.channel,
    i.test,
    i.test_group)


SELECT
  o.date,
  o.country,
  o.device,
  o.continent,
  o.channel,
  o.test,
  o.test_group,
  'order' AS event_name,
  o.total_order AS value
FROM
  total_order o
UNION ALL
SELECT
  e.date,
  e.country,
  e.device,
  e.continent,
  e.channel,
  e.test,
  e.test_group,
  e.event_name,
  e.total_event AS value
FROM
  total_event e
UNION ALL
SELECT
  s.date,
  s.country,
  s.device,
  s.continent,
  s.channel,
  s.test,
  s.test_group,
  'session' AS event_name,
  s.total_session AS value
FROM
  total_session s
UNION ALL
SELECT
  a.date,
  a.country,
  a.device,
  a.continent,
  a.channel,
  a.test,
  a.test_group,
  'new account' AS event_name,
  a.total_account AS value
FROM
  total_account a

"""


df = client.query(query).to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Kenya,desktop,Africa,Organic Search,2,1,order,1
1,2020-11-01,Puerto Rico,desktop,Americas,Organic Search,2,2,order,1
2,2020-11-02,Bahrain,desktop,Asia,Undefined,2,2,order,1
3,2020-11-02,New Zealand,desktop,Oceania,Organic Search,2,1,order,1
4,2020-11-03,Croatia,desktop,Europe,Paid Search,2,1,order,1


In [ ]:
df_group1 = df[df['test_group'] == 1].copy()
df_group2 = df[df['test_group'] == 2].copy()

In [ ]:
df_group1

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Kenya,desktop,Africa,Organic Search,2,1,order,1
3,2020-11-02,New Zealand,desktop,Oceania,Organic Search,2,1,order,1
4,2020-11-03,Croatia,desktop,Europe,Paid Search,2,1,order,1
5,2020-11-03,Algeria,desktop,Africa,Paid Search,2,1,order,1
6,2020-11-05,Lebanon,desktop,Asia,Social Search,2,1,order,1
...,...,...,...,...,...,...,...,...,...
800988,2020-11-05,Vietnam,mobile,Asia,Social Search,2,1,session_start,1
800991,2020-11-29,Vietnam,mobile,Asia,Direct,2,1,user_engagement,1
800992,2020-12-02,Vietnam,mobile,Asia,Organic Search,3,1,user_engagement,1
800993,2020-12-18,Vietnam,desktop,Asia,Paid Search,4,1,session_start,1


In [ ]:
df1_total_agg = df_group1.groupby(['event_name','test'])['value'].sum().reset_index()
df1_total_agg['category_name'] = 'Total'
df1_total_agg['test_group'] = 1

In [ ]:
order = ['category_name', 'event_name', 'test', 'value', 'test_group']
df1_total_agg =df1_total_agg[order]
df1_total_agg

,category_name,event_name,test,value,test_group
0,Total,add_payment_info,1,1988,1
1,Total,add_payment_info,2,2344,1
2,Total,add_payment_info,3,3623,1
3,Total,add_payment_info,4,3731,1
4,Total,add_shipping_info,1,3034,1
...,...,...,...,...,...
71,Total,view_promotion,4,52672,1
72,Total,view_search_results,1,3678,1
73,Total,view_search_results,2,4282,1
74,Total,view_search_results,3,5764,1


In [ ]:
df2_total_agg = df_group2.groupby(['event_name','test'])['value'].sum().reset_index()
df2_total_agg['category_name'] = 'Total'
df2_total_agg['test_group'] = 2

df2_total_agg= df2_total_agg[order]
df2_total_agg

,category_name,event_name,test,value,test_group
0,Total,add_payment_info,1,2229,2
1,Total,add_payment_info,2,2409,2
2,Total,add_payment_info,3,3697,2
3,Total,add_payment_info,4,3601,2
4,Total,add_shipping_info,1,3221,2
...,...,...,...,...,...
71,Total,view_promotion,4,51985,2
72,Total,view_search_results,1,3882,2
73,Total,view_search_results,2,4198,2
74,Total,view_search_results,3,5619,2


In [ ]:
df1_country_agg = df_group1.groupby(['country', 'event_name','test'])['value'].sum().reset_index()
df1_country_agg['country'] = df1_country_agg['country'].replace('(not set)', 'Not set country')
df1_country_agg.rename(columns={'country': 'category_name'}, inplace=True)
df1_country_agg['test_group'] = 1

df1_country_agg

,category_name,event_name,test,value,test_group
0,Not set country,add_payment_info,1,16,1
1,Not set country,add_payment_info,2,23,1
2,Not set country,add_payment_info,3,20,1
3,Not set country,add_payment_info,4,29,1
4,Not set country,add_shipping_info,1,23,1
...,...,...,...,...,...
6739,Vietnam,view_promotion,4,182,1
6740,Vietnam,view_search_results,1,17,1
6741,Vietnam,view_search_results,2,20,1
6742,Vietnam,view_search_results,3,18,1


In [ ]:
df1_continent_agg = df_group1.groupby(['continent', 'event_name','test'])['value'].sum().reset_index()
df1_continent_agg['continent'] = df1_continent_agg['continent'].replace('(not set)', 'Not set continent')
df1_continent_agg.rename(columns={'continent': 'category_name'}, inplace=True)
df1_continent_agg['test_group'] = 1

df1_continent_agg

,category_name,event_name,test,value,test_group
0,Not set continent,add_payment_info,1,7,1
1,Not set continent,add_payment_info,2,11,1
2,Not set continent,add_payment_info,3,14,1
3,Not set continent,add_payment_info,4,22,1
4,Not set continent,add_shipping_info,1,4,1
...,...,...,...,...,...
434,Oceania,view_promotion,4,513,1
435,Oceania,view_search_results,1,34,1
436,Oceania,view_search_results,2,40,1
437,Oceania,view_search_results,3,67,1


In [ ]:
df1_device_agg = df_group1.groupby(['device', 'event_name','test'])['value'].sum().reset_index()
df1_device_agg.rename(columns={'device': 'category_name'}, inplace=True)
df1_device_agg['test_group'] = 1

df1_device_agg

,category_name,event_name,test,value,test_group
0,desktop,add_payment_info,1,1130,1
1,desktop,add_payment_info,2,1314,1
2,desktop,add_payment_info,3,2087,1
3,desktop,add_payment_info,4,2211,1
4,desktop,add_shipping_info,1,1711,1
...,...,...,...,...,...
219,tablet,view_promotion,4,1275,1
220,tablet,view_search_results,1,86,1
221,tablet,view_search_results,2,91,1
222,tablet,view_search_results,3,95,1


In [ ]:
df1_channel_agg = df_group1.groupby(['channel', 'event_name','test'])['value'].sum().reset_index()
df1_channel_agg.rename(columns={'channel': 'category_name'}, inplace=True)
df1_channel_agg['test_group'] = 1

df1_channel_agg

,category_name,event_name,test,value,test_group
0,Direct,add_payment_info,1,392,1
1,Direct,add_payment_info,2,516,1
2,Direct,add_payment_info,3,861,1
3,Direct,add_payment_info,4,893,1
4,Direct,add_shipping_info,1,664,1
...,...,...,...,...,...
371,Undefined,view_promotion,4,3314,1
372,Undefined,view_search_results,1,255,1
373,Undefined,view_search_results,2,261,1
374,Undefined,view_search_results,3,382,1


In [ ]:
df2_country_agg = df_group2.groupby(['country', 'event_name','test'])['value'].sum().reset_index()
df2_country_agg['country'] = df2_country_agg['country'].replace('(not set)', 'Not set country')
df2_country_agg.rename(columns={'country': 'category_name'}, inplace=True)
df2_country_agg['test_group'] = 2

df2_country_agg

,category_name,event_name,test,value,test_group
0,Not set country,add_payment_info,1,19,2
1,Not set country,add_payment_info,2,18,2
2,Not set country,add_payment_info,3,41,2
3,Not set country,add_payment_info,4,22,2
4,Not set country,add_shipping_info,1,26,2
...,...,...,...,...,...
6736,Vietnam,view_promotion,4,215,2
6737,Vietnam,view_search_results,1,16,2
6738,Vietnam,view_search_results,2,14,2
6739,Vietnam,view_search_results,3,49,2


In [ ]:
df2_continent_agg = df_group2.groupby(['continent', 'event_name','test'])['value'].sum().reset_index()
df2_continent_agg['continent'] = df2_continent_agg['continent'].replace('(not set)', 'Not set continent')
df2_continent_agg.rename(columns={'continent': 'category_name'}, inplace=True)
df2_continent_agg['test_group'] = 2

df2_continent_agg

,category_name,event_name,test,value,test_group
0,Not set continent,add_payment_info,1,10,2
1,Not set continent,add_payment_info,2,7,2
2,Not set continent,add_payment_info,3,19,2
3,Not set continent,add_payment_info,4,8,2
4,Not set continent,add_shipping_info,1,13,2
...,...,...,...,...,...
438,Oceania,view_promotion,4,513,2
439,Oceania,view_search_results,1,29,2
440,Oceania,view_search_results,2,36,2
441,Oceania,view_search_results,3,42,2


In [ ]:
df2_device_agg = df_group2.groupby(['device', 'event_name','test'])['value'].sum().reset_index()
df2_device_agg.rename(columns={'device': 'category_name'}, inplace=True)
df2_device_agg['test_group'] = 2

df2_device_agg

,category_name,event_name,test,value,test_group
0,desktop,add_payment_info,1,1256,2
1,desktop,add_payment_info,2,1401,2
2,desktop,add_payment_info,3,2167,2
3,desktop,add_payment_info,4,2046,2
4,desktop,add_shipping_info,1,1916,2
...,...,...,...,...,...
219,tablet,view_promotion,4,1072,2
220,tablet,view_search_results,1,77,2
221,tablet,view_search_results,2,94,2
222,tablet,view_search_results,3,100,2


In [ ]:
df2_channel_agg = df_group2.groupby(['channel', 'event_name','test'])['value'].sum().reset_index()
df2_channel_agg.rename(columns={'channel': 'category_name'}, inplace=True)
df2_channel_agg['test_group'] = 2

df2_channel_agg

,category_name,event_name,test,value,test_group
0,Direct,add_payment_info,1,516,2
1,Direct,add_payment_info,2,504,2
2,Direct,add_payment_info,3,794,2
3,Direct,add_payment_info,4,831,2
4,Direct,add_shipping_info,1,716,2
...,...,...,...,...,...
372,Undefined,view_promotion,4,3383,2
373,Undefined,view_search_results,1,266,2
374,Undefined,view_search_results,2,313,2
375,Undefined,view_search_results,3,378,2


In [ ]:
all_dfs = [
    df1_total_agg,
    df1_country_agg,
    df1_continent_agg,
    df1_device_agg,
    df1_channel_agg]

group1= pd.concat(all_dfs, ignore_index=True)

print(group1)

     category_name           event_name  test  value  test_group
0            Total     add_payment_info     1   1988           1
1            Total     add_payment_info     2   2344           1
2            Total     add_payment_info     3   3623           1
3            Total     add_payment_info     4   3731           1
4            Total    add_shipping_info     1   3034           1
...            ...                  ...   ...    ...         ...
7854     Undefined       view_promotion     4   3314           1
7855     Undefined  view_search_results     1    255           1
7856     Undefined  view_search_results     2    261           1
7857     Undefined  view_search_results     3    382           1
7858     Undefined  view_search_results     4    513           1

[7859 rows x 5 columns]


In [ ]:
all_dfs = [
    df2_total_agg,
    df2_country_agg,
    df2_continent_agg,
    df2_device_agg,
    df2_channel_agg]

group2= pd.concat(all_dfs, ignore_index=True)

print(group2)

     category_name           event_name  test  value  test_group
0            Total     add_payment_info     1   2229           2
1            Total     add_payment_info     2   2409           2
2            Total     add_payment_info     3   3697           2
3            Total     add_payment_info     4   3601           2
4            Total    add_shipping_info     1   3221           2
...            ...                  ...   ...    ...         ...
7856     Undefined       view_promotion     4   3383           2
7857     Undefined  view_search_results     1    266           2
7858     Undefined  view_search_results     2    313           2
7859     Undefined  view_search_results     3    378           2
7860     Undefined  view_search_results     4    416           2

[7861 rows x 5 columns]


In [ ]:
merged_df = pd.merge(group1, group2, on=['category_name', 'event_name','test'], suffixes=('_group1', '_group2'))

merged_df

,category_name,event_name,test,value_group1,test_group_group1,value_group2,test_group_group2
0,Total,add_payment_info,1,1988,1,2229,2
1,Total,add_payment_info,2,2344,1,2409,2
2,Total,add_payment_info,3,3623,1,3697,2
3,Total,add_payment_info,4,3731,1,3601,2
4,Total,add_shipping_info,1,3034,1,3221,2
...,...,...,...,...,...,...,...
7430,Undefined,view_promotion,4,3314,1,3383,2
7431,Undefined,view_search_results,1,255,1,266,2
7432,Undefined,view_search_results,2,261,1,313,2
7433,Undefined,view_search_results,3,382,1,378,2


In [ ]:
sessions_df = merged_df[merged_df['event_name'] == 'session'].copy()

sessions_df

,category_name,event_name,test,value_group1,test_group_group1,value_group2,test_group_group2
48,Total,session,1,45362,1,45193,2
49,Total,session,2,50637,1,50244,2
50,Total,session,3,70047,1,70439,2
51,Total,session,4,105079,1,105141,2
122,Not set country,session,1,369,1,373,2
...,...,...,...,...,...,...,...
7339,Social Search,session,4,7961,1,8056,2
7410,Undefined,session,1,3336,1,3397,2
7411,Undefined,session,2,3661,1,3670,2
7412,Undefined,session,3,4636,1,4534,2


In [ ]:
merged_df = pd.merge(
    merged_df,
    sessions_df,
    on=(['category_name','test']),
    how='left',
    suffixes=('', '_session')
)

In [ ]:
merged_df

,category_name,event_name,test,value_group1,test_group_group1,value_group2,test_group_group2,event_name_session,value_group1_session,test_group_group1_session,value_group2_session,test_group_group2_session
0,Total,add_payment_info,1,1988,1,2229,2,session,45362,1,45193,2
1,Total,add_payment_info,2,2344,1,2409,2,session,50637,1,50244,2
2,Total,add_payment_info,3,3623,1,3697,2,session,70047,1,70439,2
3,Total,add_payment_info,4,3731,1,3601,2,session,105079,1,105141,2
4,Total,add_shipping_info,1,3034,1,3221,2,session,45362,1,45193,2
...,...,...,...,...,...,...,...,...,...,...,...,...
7430,Undefined,view_promotion,4,3314,1,3383,2,session,5716,1,5862,2
7431,Undefined,view_search_results,1,255,1,266,2,session,3336,1,3397,2
7432,Undefined,view_search_results,2,261,1,313,2,session,3661,1,3670,2
7433,Undefined,view_search_results,3,382,1,378,2,session,4636,1,4534,2


In [ ]:
columns_to_drop = ['test_group_group1_session', 'test_group_group2_session']
merged_df.drop(columns=columns_to_drop, inplace=True)

merged_df['ratio_group1'] = merged_df['value_group1'] / merged_df['value_group1_session']
merged_df['ratio_group2'] = merged_df['value_group2'] / merged_df['value_group2_session']

merged_df

,category_name,event_name,test,value_group1,test_group_group1,value_group2,test_group_group2,event_name_session,value_group1_session,value_group2_session,ratio_group1,ratio_group2
0,Total,add_payment_info,1,1988,1,2229,2,session,45362,45193,0.043825,0.049322
1,Total,add_payment_info,2,2344,1,2409,2,session,50637,50244,0.04629,0.047946
2,Total,add_payment_info,3,3623,1,3697,2,session,70047,70439,0.051722,0.052485
3,Total,add_payment_info,4,3731,1,3601,2,session,105079,105141,0.035507,0.034249
4,Total,add_shipping_info,1,3034,1,3221,2,session,45362,45193,0.066884,0.071272
...,...,...,...,...,...,...,...,...,...,...,...,...
7430,Undefined,view_promotion,4,3314,1,3383,2,session,5716,5862,0.579776,0.577107
7431,Undefined,view_search_results,1,255,1,266,2,session,3336,3397,0.076439,0.078304
7432,Undefined,view_search_results,2,261,1,313,2,session,3661,3670,0.071292,0.085286
7433,Undefined,view_search_results,3,382,1,378,2,session,4636,4534,0.082399,0.08337


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
p_values = []

for index, row in merged_df.iterrows():
    count = [row['value_group1'], row['value_group2']]
    nobs = [row['value_group1_session'], row['value_group2_session']]
    zstat, pvalue = proportions_ztest(count=count, nobs=nobs, alternative='two-sided')
    p_values.append(pvalue)

merged_df['pvalue'] = p_values

print(merged_df)


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/proportion.py:1024: RuntimeWarning: invalid value encountered in sqrt
  std_diff = np.sqrt(var_)
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: divide by zero encountered in scalar divide
  zstat = value / std


     category_name           event_name  test  value_group1  \
0            Total     add_payment_info     1          1988   
1            Total     add_payment_info     2          2344   
2            Total     add_payment_info     3          3623   
3            Total     add_payment_info     4          3731   
4            Total    add_shipping_info     1          3034   
...            ...                  ...   ...           ...   
7430     Undefined       view_promotion     4          3314   
7431     Undefined  view_search_results     1           255   
7432     Undefined  view_search_results     2           261   
7433     Undefined  view_search_results     3           382   
7434     Undefined  view_search_results     4           513   

      test_group_group1  value_group2  test_group_group2 event_name_session  \
0                     1          2229                  2            session   
1                     1          2409                  2            session   
2     

In [ ]:
merged_df['significant'] = merged_df['pvalue'] < 0.05

merged_df

,category_name,event_name,test,value_group1,test_group_group1,value_group2,test_group_group2,event_name_session,value_group1_session,value_group2_session,ratio_group1,ratio_group2,pvalue,significant
0,Total,add_payment_info,1,1988,1,2229,2,session,45362,45193,0.043825,0.049322,0.000087,True
1,Total,add_payment_info,2,2344,1,2409,2,session,50637,50244,0.04629,0.047946,0.214608,False
2,Total,add_payment_info,3,3623,1,3697,2,session,70047,70439,0.051722,0.052485,0.520112,False
3,Total,add_payment_info,4,3731,1,3601,2,session,105079,105141,0.035507,0.034249,0.116158,False
4,Total,add_shipping_info,1,3034,1,3221,2,session,45362,45193,0.066884,0.071272,0.009226,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430,Undefined,view_promotion,4,3314,1,3383,2,session,5716,5862,0.579776,0.577107,0.771209,False
7431,Undefined,view_search_results,1,255,1,266,2,session,3336,3397,0.076439,0.078304,0.774541,False
7432,Undefined,view_search_results,2,261,1,313,2,session,3661,3670,0.071292,0.085286,0.025740,True
7433,Undefined,view_search_results,3,382,1,378,2,session,4636,4534,0.082399,0.08337,0.866030,False


In [ ]:
merged_df.to_csv('ab_test_results.csv', index=False)

#Дашборд

[https://public.tableau.com/app/profile/denys.diakiv/viz/ABTestPerformanceOverview/ABTestPerformanceOverview?publish=yes](https://public.tableau.com/app/profile/denys.diakiv/viz/ABTestPerformanceOverview/ABTestPerformanceOverview?publish=yes)

#CSV-файл

[https://drive.google.com/file/d/1OK0hIiTXqO5dLJyWKQHBoigXBSHM6lZD/view?usp=sharing](https://drive.google.com/file/d/1OK0hIiTXqO5dLJyWKQHBoigXBSHM6lZD/view?usp=sharing)

В результаті аналізу ми отримали повну картину змін за варіантами A/B-тесту, розбиту за кожним окремим показником і кожним некобінованим розрізом, таким як країна, пристрій, континент, канал, тощо.

Для кожного з цих сегментів ми можемо:

Порівняти показники: Зіставити коефіцієнти конверсії (або інші показники) між контрольною та тестовою групами.

Перевірити статистичну значущість: Завдяки пропорційному Z-тесту, ми можемо визначити, чи є виявлена різниця між групами випадковою, чи вона дійсно є результатом змін, внесених у тестовий варіант.